In [17]:
!pip install matplotlib pillow

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


You should consider upgrading via the 'C:\Users\dyctw\.pyenv\pyenv-win\versions\3.10.5\python.exe -m pip install --upgrade pip' command.


In [6]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
from PIL import Image

# 設定波參數
time_steps = 50
amplitude = 0.5
wave_length = 2 * np.pi
k = 2 * np.pi / wave_length
omega = k

# 建立網格方塊 (增加x範圍到20)
Nx, Ny, Nz = 40, 4, 4
x = np.linspace(0, 20, Nx)
y = np.linspace(-1, 1, Ny)
z = np.linspace(-1, 1, Nz)
X0, Y0, Z0 = np.meshgrid(x, y, z, indexing='ij')

def create_wireframe(ax, Xd, Yd, Zd, color='orange'):
    for j in range(Yd.shape[1]):
        for k_ in range(Zd.shape[2]):
            ax.plot(Xd[:, j, k_], Yd[:, j, k_], Zd[:, j, k_], color=color, linewidth=0.8)
    for i in range(Xd.shape[0]):
        for k_ in range(Zd.shape[2]):
            ax.plot(Xd[i, :, k_], Yd[i, :, k_], Zd[i, :, k_], color=color, linewidth=0.8)
    for i in range(Xd.shape[0]):
        for j in range(Yd.shape[1]):
            ax.plot(Xd[i, j, :], Yd[i, j, :], Zd[i, j, :], color=color, linewidth=0.8)

# 建立存放臨時影格圖片的資料夾
if not os.path.exists('frames'):
    os.makedirs('frames')

# 產生 P 波 PNG 影格
fig_p = plt.figure(figsize=(10, 6))
ax_p = fig_p.add_subplot(111, projection='3d')
fig_p.patch.set_alpha(0)
ax_p.set_facecolor('none')
for frame in range(time_steps):
    ax_p.cla()
    ax_p.set_facecolor('none')
    ax_p.axis('off')
    ax_p.set_xlim(0, 20)
    ax_p.set_ylim(-1, 1)
    ax_p.set_zlim(-1, 1)
    ax_p.set_title('P-wave', fontsize=16)
    t = frame
    displacement = amplitude * np.sin(k * X0 - omega * t)
    Xp = X0 + displacement
    create_wireframe(ax_p, Xp, Y0, Z0)
    plt.savefig(f'frames/p_wave_{frame:03d}.png', transparent=True, dpi=100)
plt.close(fig_p)

# 產生 S 波 PNG 影格
fig_s = plt.figure(figsize=(10, 6))
ax_s = fig_s.add_subplot(111, projection='3d')
fig_s.patch.set_alpha(0)
ax_s.set_facecolor('none')
for frame in range(time_steps):
    ax_s.cla()
    ax_s.set_facecolor('none')
    ax_s.axis('off')
    ax_s.set_xlim(0, 20)
    ax_s.set_ylim(-1, 1)
    ax_s.set_zlim(-1, 1)
    ax_s.set_title('S-wave', fontsize=16)
    t = frame
    displacement = amplitude * np.sin(k * X0 - omega * t)
    Zs = Z0 + displacement
    create_wireframe(ax_s, X0, Y0, Zs)
    plt.savefig(f'frames/s_wave_{frame:03d}.png', transparent=True, dpi=100)
plt.close(fig_s)

def pngs_to_gif(pattern, output_gif, skip=0):
    frames_list = sorted([f for f in os.listdir('frames') if f.startswith(pattern)])
    frames_list = frames_list[skip:]  # 略過前幾幀

    images = []
    for f in frames_list:
        img = Image.open(os.path.join('frames', f)).convert("RGBA")

        # 將透明疊放到白色背景上 (確保最終背景為白色)
        white_bg = Image.new("RGBA", img.size, (255, 255, 255, 255))
        white_bg.alpha_composite(img)

        # 轉為RGB再量化為P模式，確保有合適的palette
        p_img = white_bg.convert("RGB").quantize(colors=256, method=Image.MEDIANCUT)

        # 找出最接近白色的調色盤索引
        palette = p_img.getpalette()
        closest_white = 0
        min_dist = 255**2*3
        for i in range(256):
            r, g, b = palette[3*i], palette[3*i+1], palette[3*i+2]
            dist = (r-255)**2+(g-255)**2+(b-255)**2
            if dist < min_dist:
                min_dist = dist
                closest_white = i

        # 每個frame都存成PIL Image物件，後續一次存GIF時指定transparency
        p_img.info['transparency'] = closest_white
        images.append(p_img)

    # 將 images 合成 GIF，指定透明索引
    images[0].save(
        output_gif,
        save_all=True,
        append_images=images[1:],
        loop=0,
        duration=100,
        disposal=2,
        transparency=images[0].info['transparency']
    )

# 將 P 波影格組合成GIF，略過前5幀
pngs_to_gif('p_wave_', 'p_wave_3d_block_fixed.gif', skip=5)

# 將 S 波影格組合成GIF，略過前5幀
pngs_to_gif('s_wave_', 's_wave_3d_block_fixed.gif', skip=5)

print("已產生 p_wave_3d_block_fixed.gif 與 s_wave_3d_block_fixed.gif，使用量化與單一透明色避免 palette 問題。")


已產生 p_wave_3d_block_fixed.gif 與 s_wave_3d_block_fixed.gif，使用量化與單一透明色避免 palette 問題。
